In [ ]:
import pandas as pd
import numpy as np
from numpy import vstack
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCELoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

In [ ]:
RC = pd.read_csv(r'../input/room-occupancy/file.csv')

In [ ]:
RC.head()

In [ ]:
RC['Occupancy'].value_counts()

In [ ]:
RC.info()

In [ ]:
RC.iloc[ : , 0 : 5]

In [ ]:
RC.iloc[ : , 5 : 6]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

class CSVDataset(Dataset):
    
    def __init__(self, path):
        df = pd.read_csv(path)
        
        K = df['Temperature'].values
        K = K.reshape(-1, 1)
        scaler = MinMaxScaler()
        df['Temperature'] = scaler.fit_transform(K)
        
        K = df['Humidity'].values
        K = K.reshape(-1, 1)
        scaler = MinMaxScaler()
        df['Humidity'] = scaler.fit_transform(K)
        
        K = df['Light'].values
        K = K.reshape(-1, 1)
        scaler = MinMaxScaler()
        df['Light'] = scaler.fit_transform(K)
        
        K = df['CO2'].values
        K = K.reshape(-1, 1)
        scaler = MinMaxScaler()
        df['CO2'] = scaler.fit_transform(K)
        
        K = df['Light'].values
        K = K.reshape(-1, 1)
        scaler = MinMaxScaler()
        df['Light'] = scaler.fit_transform(K)
        
        K = df['HumidityRatio'].values
        K = K.reshape(-1, 1)
        scaler = MinMaxScaler()
        df['HumidityRatio'] = scaler.fit_transform(K)
        
        
        
        print(df.head())
        
        df = df.values
        self.X = df[: , 0 : 5]
        self.y = df[: , 5 : 6]
        self.X = self.X.astype('float32')
        print('initX', len(self.X))
        print('initShapeX', (self.X).shape)
        self.y = self.y.astype('float32')
        self.y = self.y.reshape((len(self.y), 1))
        print('inity', len(self.y))
        print('initShapey', (self.y).shape)
        
    def __len__(self):
        return (len(self.X))
    
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]
    
    def get_splits(self, n_test = 0.33):
        test_size = round(n_test * len(self.X))
        print('SplitSizeTest', test_size)
        train_size = len(self.X) - test_size
        print('SplitSizeTrain', train_size)
        return random_split(self, [train_size, test_size])

In [ ]:
class Model(Module):
    
    def __init__(self, n_inputs):
        super(Model, self).__init__()
        self.hidden1 = Linear(n_inputs, 16)
        kaiming_uniform_(self.hidden1.weight, nonlinearity = 'relu')
        self.act1 = ReLU()
        self.hidden2 = Linear(16, 8)
        kaiming_uniform_(self.hidden2.weight, nonlinearity = 'relu')
        self.act2 = ReLU()
        self.hidden3 = Linear(8, 1)
        xavier_uniform_(self.hidden3.weight)
        self.act3 = Sigmoid()
        
        
    def forward(self, X):
        X = self.hidden1(X)
        X = self.act1(X)
        X = self.hidden2(X)
        X = self.act2(X)
        X = self.hidden3(X)
        X = self.act3(X)
        
        return X    

In [ ]:
 def data(path):
    dataset = CSVDataset(path)
    train, test = dataset.get_splits()
    train_dl = DataLoader(train, batch_size = 32, shuffle = True)
    print('dataFunctionTrain', len(train_dl))
    test_dl = DataLoader(test, batch_size = 1024, shuffle = False)
    print('dataFunctionTest', len(test_dl))
        
    return train_dl, test_dl

In [ ]:
def train_model(train_dl, model):
    criterion = BCELoss()
    optimizer = SGD(model.parameters(), lr = 0.01, momentum = 0.9)
    for epoch in range(100):
        for i, (inputs, targets) in enumerate(train_dl):
            optimizer.zero_grad()
            yhat = model(inputs)
            loss = criterion(yhat, targets)
            loss.backward()
            optimizer.step()

In [ ]:
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        yhat = model(inputs)
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        yhat = yhat.round()
        predictions.append(yhat)
        actuals.append(actual)
    
    predictions, actuals = vstack(predictions), vstack(actuals)
    acc = accuracy_score(actuals, predictions)
    return acc

In [ ]:
path = '../input/room-occupancy/file.csv'

train_dl, test_dl = data(path)
print(len(train_dl), len(test_dl))
model = Model(5)
train_model(train_dl, model)
acc = evaluate_model(test_dl, model)
print('Accuracy: %0.3f' % acc)